In [45]:
import import_ipynb
import preprocess_model
import numpy as np
import math

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM, Reshape


In [77]:
df = pd.read_csv('./data/new_data_asia.csv')

In [78]:
numerical_features = ['School enrollment, primary', 'School enrollment, tertiary', 'Primary completion rate', 'Year']
categorical_features = ['Country']
target = 'Literacy rate'

# Drop rows with missing target values
df.dropna(subset=[target], inplace=True)

In [105]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from keras.models import Model
from keras.layers import Input, SimpleRNN, Dense, Concatenate
import numpy as np

In [108]:
# Label Encoding for categorical features
label_encoders = {}
for feature in categorical_features:
    label_encoders[feature] = LabelEncoder()
    df[feature] = label_encoders[feature].fit_transform(df[feature])

# One-Hot Encoding for categorical features
one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_cat_encoded = one_hot_encoder.fit_transform(df[categorical_features])

# Standard Scaling for numerical features
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(df[numerical_features])

# Concatenate numerical and categorical features
X = np.concatenate((X_cat_encoded, X_num_scaled), axis=1)
y = df[target]

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Split numerical and categorical features in train and validation sets
X_cat_train, X_num_train = X_train[:, :len(categorical_features)], X_train[:, len(categorical_features):]
X_cat_val, X_num_val = X_val[:, :len(categorical_features)], X_val[:, len(categorical_features):]

# Reshape the numerical data if necessary
if len(X_num_train.shape) == 2:
    X_num_train = np.expand_dims(X_num_train, axis=2)
    X_num_val = np.expand_dims(X_num_val, axis=2)

# Define model architecture
num_input = Input(shape=(X_num_train.shape[1], X_num_train.shape[2]))
rnn_layer = SimpleRNN(64, activation='relu')(num_input)

cat_input = Input(shape=(X_cat_train.shape[1],))
cat_output = Dense(64, activation='relu')(cat_input)

concatenated = Concatenate()([rnn_layer, cat_output])
output = Dense(1)(concatenated)

model = Model(inputs=[num_input, cat_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit([X_num_train, X_cat_train], y_train, epochs=200, batch_size=64,
                    validation_data=([X_num_val, X_cat_val], y_val), verbose=1)

# Plot the training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  ----------


Epoch 1/200
4/4 [==============================] - 2s 120ms/step - loss: 8921.1826 - val_loss: 9169.8701
Epoch 2/200
4/4 [==============================] - 0s 17ms/step - loss: 8894.3057 - val_loss: 9143.6152
Epoch 3/200
4/4 [==============================] - 0s 17ms/step - loss: 8865.4502 - val_loss: 9113.2412
Epoch 4/200
4/4 [==============================] - 0s 18ms/step - loss: 8831.8477 - val_loss: 9067.7988
Epoch 5/200
4/4 [==============================] - 0s 18ms/step - loss: 8768.9043 - val_loss: 8904.1172
Epoch 6/200
4/4 [==============================] - 0s 17ms/step - loss: 8381.5918 - val_loss: 6848.0229
Epoch 7/200
4/4 [==============================] - 0s 18ms/step - loss: 4065.0537 - val_loss: 871.3710
Epoch 8/200
4/4 [==============================] - 0s 17ms/step - loss: 783.1224 - val_loss: 1167.0432
Epoch 9/200
4/4 [==============================] - 0s 17ms/step - loss: 650.3815 - val_loss: 812.9379
Epoch 10/200
4/4 [==============================] - 0s 17ms/step - 